In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, GridSearchCV, cross_val_score
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error, r2_score, mean_absolute_error
import optuna

/usr/local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
seed = 100

df = pd.read_pickle("../data/RDKit/rdkit_only_valid_smiles_qm9.pkl")
# Check the info of the dataframe, especially important is the dtypes of the columns
# Split features and target
X = df.drop(columns=['gaps', 'SMILES']) # Drop the target variable and the SMILES objects
y = df['gaps']

In [4]:
df.info()
X.info()
X.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129012 entries, 0 to 129011
Columns: 219 entries, MaxAbsEStateIndex to gaps
dtypes: float64(108), int64(110), object(1)
memory usage: 215.6+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129012 entries, 0 to 129011
Columns: 217 entries, MaxAbsEStateIndex to fr_urea
dtypes: float64(107), int64(110)
memory usage: 213.6 MB


MaxAbsEStateIndex    float64
MaxEStateIndex       float64
MinAbsEStateIndex    float64
MinEStateIndex       float64
qed                  float64
                      ...   
fr_thiazole            int64
fr_thiocyan            int64
fr_thiophene           int64
fr_unbrch_alkane       int64
fr_urea                int64
Length: 217, dtype: object

In [5]:
constant_cols = [col for col in X.columns if X[col].nunique() <= 1]
len(constant_cols), constant_cols

(23,
 ['SMR_VSA8',
  'SlogP_VSA12',
  'SlogP_VSA9',
  'EState_VSA11',
  'VSA_EState10',
  'fr_C_S',
  'fr_SH',
  'fr_azide',
  'fr_barbitur',
  'fr_benzodiazepine',
  'fr_diazo',
  'fr_dihydropyridine',
  'fr_isocyan',
  'fr_isothiocyan',
  'fr_phos_acid',
  'fr_phos_ester',
  'fr_prisulfonamd',
  'fr_sulfide',
  'fr_sulfonamd',
  'fr_sulfone',
  'fr_thiazole',
  'fr_thiocyan',
  'fr_thiophene'])

In [34]:
# --- Split ---
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=seed
)

# --- LightGBM Dataset ---
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_valid, label=y_valid)

# --- Parameters ---
params = {
    "objective": "regression_l1",  # MAE optimization
    "metric": "l1",                # MAE evaluation
    "boosting_type": "gbdt",
    "learning_rate": 0.05,
    "num_leaves": 64,
    "max_depth": -1,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "seed": seed,
    "force_col_wise": True
}

# --- Train ---
model = lgb.train(
    params,
    train_set=train_data,
    valid_sets=valid_data,
    num_boost_round=10000
)

# --- Predict ---
y_pred = model.predict(X_valid, num_iteration=model.best_iteration)



KeyboardInterrupt: 

In [ ]:
# --- Evaluate ---
rmse = root_mean_squared_error(y_valid, y_pred)
mae  = mean_absolute_error(y_valid, y_pred)
r2   = r2_score(y_valid, y_pred)

print(f"MAE:  {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R²:   {r2:.4f}")


MAE:  0.1647
RMSE: 0.2569
R²:   0.9598


In [ ]:
from sklearn.model_selection import StratifiedKFold

num_bins = 10  # 10 bins for deciles
gap_bins_outer = pd.qcut(y, q=num_bins, labels=False)
print(gap_bins_outer)
# Different seed for outer and inner CV
outer_seed = 42
inner_seed = 123

# Outer CV (train/test), 80/20 split of total dataset
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=outer_seed)

for outer_fold_idx, (train_idx, test_idx) in enumerate(outer_cv.split(X, gap_bins_outer)):

    #Split into train/test
    X_train_outer, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train_outer, y_test = y.iloc[train_idx], y.iloc[test_idx]

    outer_fold = {
        "outer_fold_idx": outer_fold_idx,
        "X_test": X_test,
        "y_test": y_test,
        "inner_folds": []
    }
    
    # Create second binning that only works on the outer train data
    gap_bins_inner = pd.qcut(y_train_outer, q=num_bins, labels=False)

    # Inner CV (train/val), 72/8 split of total dataset
    inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=inner_seed)


    for inner_fold_idx, (inner_train_idx, val_idx) in enumerate(inner_cv.split(X_train_outer, gap_bins_inner)):

        # Split into train/validation
        X_train_inner, X_val = X_train_outer.iloc[inner_train_idx], X_train_outer.iloc[val_idx]
        y_train_inner, y_val = y_train_outer.iloc[inner_train_idx], y_train_outer.iloc[val_idx]

        inner_fold = {
            "inner_fold_idx": inner_fold_idx,
            "X_train": X_train_inner,
            "y_train": y_train_inner,
            "X_val": X_val,
            "y_val": y_val
        }

        # Append inner fold to outer fold
        outer_fold["inner_folds"].append(inner_fold)

0         9
1         9
2         9
3         9
4         9
         ..
129007    9
129008    9
129009    7
129010    8
129011    8
Name: gaps, Length: 129012, dtype: int64


In [11]:
import optuna
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error


# ------------------------------------------------------
# Create stratification bins for regression
# ------------------------------------------------------
def make_stratified_bins(y, n_bins=10):
    return pd.qcut(y, q=n_bins, labels=False, duplicates="drop")


# ------------------------------------------------------
# FAST Objective for Optuna
# ------------------------------------------------------
def objective(trial, X, y, y_bins):
    params = {
        "objective": "regression",
        "metric": "mae",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "num_leaves": trial.suggest_int("num_leaves", 16, 256),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.7, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.7, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 5),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 100),
    }

    # 2-fold stratified inner CV
    inner_cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
    maes = []

    for train_idx, val_idx in inner_cv.split(X, y_bins):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        train_data = lgb.Dataset(X_train, y_train)
        val_data = lgb.Dataset(X_val, y_val)

        model = lgb.train(
            params,
            train_data,
            valid_sets=[val_data],
            num_boost_round=1000
        )

        preds = model.predict(X_val)
        maes.append(mean_absolute_error(y_val, preds))

    return np.mean(maes)


# ------------------------------------------------------
# FAST Nested CV with stratified folds
# ------------------------------------------------------
def fast_nested_cv_stratified(X, y, n_trials=10):
    # Make stratification bins
    y_bins = make_stratified_bins(y, n_bins=10)

    # 3-fold stratified outer CV
    outer_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

    outer_maes = []
    best_params_list = []

    fold_num = 1
    for train_idx, test_idx in outer_cv.split(X, y_bins):
        print(f"\n=== Outer Fold {fold_num} ===")

        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        # Recompute bins for inner fold (important!)
        y_bins_inner = make_stratified_bins(y_train, n_bins=10)

        # Inner optimization
        study = optuna.create_study(direction="minimize")
        study.optimize(
            lambda t: objective(t, X_train, y_train, y_bins_inner),
            n_trials=n_trials,
            show_progress_bar=False#,n_jobs=4
        )

        best_params = study.best_params
        best_params["objective"] = "regression"
        best_params["metric"] = "mae"
        best_params["verbosity"] = -1
        best_params_list.append(best_params)

        # Train on full outer training fold
        train_set = lgb.Dataset(X_train, y_train)
        val_set = lgb.Dataset(X_test, y_test)

        model = lgb.train(
            best_params,
            train_set,
            valid_sets=[val_set],
            num_boost_round=1000
        )

        preds = model.predict(X_test)
        mae = mean_absolute_error(y_test, preds)

        print(f"Fold {fold_num} MAE: {mae:.4f}")
        outer_maes.append(mae)

        fold_num += 1

    return outer_maes, best_params_list


In [12]:
#early stopping and n_jobs in optuna not supported in lightgbm train API
outer_maes, best_params_list = fast_nested_cv_stratified(X, y, n_trials=100)
print("\n=== Summary of Outer Fold MAEs ===")
for i, mae in enumerate(outer_maes, 1):
    print(f"Fold {i} MAE: {mae:.4f}")
print(f"Overall MAE: {np.mean(outer_maes):.4f} ± {np.std(outer_maes):.4f}")

print(f"BEST PARAMETERS FROM EACH OUTER FOLD:")
for i, params in enumerate(best_params_list, 1):
    print(f"Outer Fold {i}: {params}")


=== Outer Fold 1 ===


[I 2025-11-21 18:41:07,454] A new study created in memory with name: no-name-83395b7e-27f3-4476-a79f-dda476a1c189
[I 2025-11-21 18:43:10,931] Trial 0 finished with value: 0.24956466112087378 and parameters: {'num_leaves': 48, 'learning_rate': 0.004424204408818223, 'feature_fraction': 0.768247336975975, 'bagging_fraction': 0.7874459562434635, 'bagging_freq': 2, 'min_data_in_leaf': 85}. Best is trial 0 with value: 0.24956466112087378.
[I 2025-11-21 18:47:20,380] Trial 1 finished with value: 0.3011652528156653 and parameters: {'num_leaves': 150, 'learning_rate': 0.0018524896020293677, 'feature_fraction': 0.7726308760437448, 'bagging_fraction': 0.960347915053823, 'bagging_freq': 4, 'min_data_in_leaf': 22}. Best is trial 0 with value: 0.24956466112087378.
[I 2025-11-21 18:54:28,711] Trial 2 finished with value: 0.17874439562436847 and parameters: {'num_leaves': 221, 'learning_rate': 0.008007817973037908, 'feature_fraction': 0.7987437615028767, 'bagging_fraction': 0.9121175541987646, 'baggin

Fold 1 MAE: 0.1493

=== Outer Fold 2 ===


[I 2025-11-21 23:37:03,237] A new study created in memory with name: no-name-75fb9f31-955c-483b-a830-8eff2d5a420a
[I 2025-11-21 23:38:33,574] Trial 0 finished with value: 0.16263261885683708 and parameters: {'num_leaves': 108, 'learning_rate': 0.0573149975936314, 'feature_fraction': 0.8411436931231845, 'bagging_fraction': 0.7269767781799124, 'bagging_freq': 1, 'min_data_in_leaf': 19}. Best is trial 0 with value: 0.16263261885683708.
[I 2025-11-21 23:41:39,236] Trial 1 finished with value: 0.158801159961671 and parameters: {'num_leaves': 235, 'learning_rate': 0.02756020410881636, 'feature_fraction': 0.8219261677981117, 'bagging_fraction': 0.9695848338913462, 'bagging_freq': 3, 'min_data_in_leaf': 13}. Best is trial 1 with value: 0.158801159961671.
[I 2025-11-21 23:44:03,562] Trial 2 finished with value: 0.15958359779424985 and parameters: {'num_leaves': 194, 'learning_rate': 0.0398245238022969, 'feature_fraction': 0.8250455478104205, 'bagging_fraction': 0.7930420862707847, 'bagging_freq

Fold 2 MAE: 0.1482

=== Outer Fold 3 ===


[I 2025-11-22 03:50:43,846] A new study created in memory with name: no-name-95b45747-0d60-4d23-a6a9-189c7100b767
[I 2025-11-22 03:52:15,529] Trial 0 finished with value: 0.25200509564817675 and parameters: {'num_leaves': 47, 'learning_rate': 0.0042523495187018695, 'feature_fraction': 0.9172132716650746, 'bagging_fraction': 0.7257100875325353, 'bagging_freq': 1, 'min_data_in_leaf': 21}. Best is trial 0 with value: 0.25200509564817675.
[I 2025-11-22 03:54:29,592] Trial 1 finished with value: 0.22270423841932754 and parameters: {'num_leaves': 104, 'learning_rate': 0.004245632422092102, 'feature_fraction': 0.9003662974466862, 'bagging_fraction': 0.7607576122259657, 'bagging_freq': 3, 'min_data_in_leaf': 64}. Best is trial 1 with value: 0.22270423841932754.
[I 2025-11-22 03:56:27,304] Trial 2 finished with value: 0.2954240815369032 and parameters: {'num_leaves': 65, 'learning_rate': 0.0022711779895809568, 'feature_fraction': 0.7937939251095862, 'bagging_fraction': 0.9907517326988058, 'bagg

Fold 3 MAE: 0.1483

=== Summary of Outer Fold MAEs ===
Fold 1 MAE: 0.1493
Fold 2 MAE: 0.1482
Fold 3 MAE: 0.1483
Overall MAE: 0.1486 ± 0.0005
BEST PARAMETERS FROM EACH OUTER FOLD:
Outer Fold 1: {'num_leaves': 237, 'learning_rate': 0.05283272421443799, 'feature_fraction': 0.7285176273657754, 'bagging_fraction': 0.9888976341447224, 'bagging_freq': 5, 'min_data_in_leaf': 10, 'objective': 'regression', 'metric': 'mae', 'verbosity': -1}
Outer Fold 2: {'num_leaves': 256, 'learning_rate': 0.056054176083826915, 'feature_fraction': 0.7112863838847365, 'bagging_fraction': 0.9602787441530835, 'bagging_freq': 5, 'min_data_in_leaf': 15, 'objective': 'regression', 'metric': 'mae', 'verbosity': -1}
Outer Fold 3: {'num_leaves': 247, 'learning_rate': 0.05328143741461067, 'feature_fraction': 0.7835672031599564, 'bagging_fraction': 0.9145047641787873, 'bagging_freq': 3, 'min_data_in_leaf': 29, 'objective': 'regression', 'metric': 'mae', 'verbosity': -1}
